In [251]:
import ssl
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import re
import pandas as pd 
import random
import numpy as np

In [94]:
def get_links(pages = 1):
    #gets the links from trulia properties page 
    # if length of list is zero, captcha blocked request so use a new VPN location 
    #pages: number of pages of trula to scrape
    #returns: list of links
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    i = 0
    all_property_links = []
    while i < pages:
        page_property_links = []
        url = 'https://www.trulia.com/NY/New_York/' + "%s_p/"%str(i)
        req = Request(url, headers={"User-Agent": 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage , 'html.parser')
        for link in soup.findAll('a'):
            l = str(link.get('href'))
            if sum(map(str.isdigit, l[-17:])) == 15: page_property_links.append("https://www.trulia.com" + l)
        page_property_links = page_property_links[::2]
        all_property_links.extend(page_property_links)
        print("finishing loop" + " " + str(i+1))
        i += 1
        print(len(all_property_links))
        time.sleep(random.randint(5,10))
        
    return(all_property_links)

In [232]:
links = get_links(pages = 35)
feature_dict_list = {}
all_features = []
all_links = []

finishing loop 1
57
finishing loop 2
114
finishing loop 3
150
finishing loop 4
187
finishing loop 5
224
finishing loop 6
262
finishing loop 7
300
finishing loop 8
340
finishing loop 9
377
finishing loop 10
414
finishing loop 11
448
finishing loop 12
486
finishing loop 13
519
finishing loop 14
553
finishing loop 15
588
finishing loop 16
624
finishing loop 17
660
finishing loop 18
696
finishing loop 19
733
finishing loop 20
768
finishing loop 21
834
finishing loop 22
891
finishing loop 23
948
finishing loop 24
1005
finishing loop 25
1062
finishing loop 26
1119
finishing loop 27
1176
finishing loop 28
1233
finishing loop 29
1290
finishing loop 30
1347
finishing loop 31
1404
finishing loop 32
1461
finishing loop 33
1518
finishing loop 34
1575
finishing loop 35
1632


In [234]:
for index,url in enumerate(links): #iterates through all URLs from the get_links func
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()  
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    feature_dict = {}

    for sou in soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"}):
        clean_data = str(sou).split("gmLKqq\">")
        clean_data = clean_data[1].split("</span>")[0]
        dict_info = clean_data.split(':')
        if len(dict_info)>1:
            if dict_info[0] == "Lot Area":
                if dict_info[1].split(' ')[2] == 'acres':
                    dict_info[1] = float(dict_info[1].split(' ')[1])*43560
                else:
                    dict_info[1] = dict_info[1].split(' ')[1]
            feature_dict[dict_info[0]] = dict_info[1]

        for bed_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bedrooms"}):
            number_beds = re.findall(r"[0-9]+\s[bB]eds*", str(bed_html))
            if number_beds:
                feature_dict["beds"] = int(number_beds[0].split(" ")[0])

        for bath_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bathrooms"}):
            number_baths = re.findall(r"[0-9]+\s[bB]aths*", str(bath_html))
            if number_baths:
                feature_dict["baths"] = int(number_baths[0].split(" ")[0])

        for floor_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-floorspace"}):
            floor_footage = re.findall(r"[0-9]+,[0-9]+\ssqft*", str(floor_html))
            if floor_footage:
                feature_dict["floor_space"] = int(floor_footage[0].split(" ")[0].replace(",", ""))
        for meta in soup.findAll('meta', attrs={'name': 'description'}):
            if meta['content']:
                feature_dict['price'] = meta['content'].split("$")[1].split(".")[0]
           
        
        feature_dict["Zip Code"] = url.split("-")[-3]
        

    all_features.append(feature_dict);all_links.append(url)
    print(str(index)+"/"+str(len(links)))
    time.sleep(random.randint(5,10))

0/1632
1/1632
2/1632
3/1632
4/1632
5/1632
6/1632
7/1632
8/1632
9/1632
10/1632
11/1632
12/1632
13/1632
14/1632
15/1632
16/1632
17/1632
18/1632
19/1632
20/1632
21/1632
22/1632
23/1632
24/1632
25/1632
26/1632
27/1632
28/1632
29/1632
30/1632
31/1632
32/1632
33/1632
34/1632
35/1632
36/1632
37/1632
38/1632
39/1632
40/1632
41/1632
42/1632
43/1632
44/1632
45/1632
46/1632
47/1632
48/1632
49/1632
50/1632
51/1632
52/1632
53/1632
54/1632
55/1632
56/1632
57/1632
58/1632
59/1632
60/1632
61/1632
62/1632
63/1632
64/1632
65/1632
66/1632
67/1632
68/1632
69/1632
70/1632
71/1632
72/1632
73/1632
74/1632
75/1632
76/1632
77/1632
78/1632
79/1632
80/1632
81/1632
82/1632
83/1632
84/1632
85/1632
86/1632
87/1632
88/1632
89/1632
90/1632
91/1632
92/1632
93/1632
94/1632
95/1632
96/1632
97/1632
98/1632
99/1632
100/1632
101/1632
102/1632
103/1632
104/1632
105/1632
106/1632
107/1632
108/1632
109/1632
110/1632
111/1632
112/1632
113/1632
114/1632
115/1632
116/1632
117/1632
118/1632
119/1632
120/1632
121/1632
122/1632
123

In [284]:
#making data frame of features
df = pd.DataFrame.from_dict(all_features)
df.to_csv("Raw_Property_Features_Data.csv")

In [286]:
#converting df columsn to correct types and fitlering out any cols with excess nan values
df["links"] = all_links
df = df[["links","Stories","beds","baths","price","Zip Code","Days on Market","Year Built","Lot Area","Property Type"]]
df['Stories'] = pd.to_numeric(df['Stories'])
df['price'] = pd.to_numeric(df.price.str.split(',').str.join(''))
df['Year Built'] = pd.to_numeric(df['Year Built'])
df["Lot Area"] = df["Lot Area"].astype(str)
df.rename(columns={"Lot Area": "Lot Area (sqft)"})

#regex days ion market to extract just the numeric value
search = []    
for values in df['Days on Market']:
    if type(values) == str:
        print
        search.append(re.search(r'\d+', str(values)).group())
    else:
        search.append('nan')
df['Days on Market'] = search

In [288]:
#filtering for desired data - rest of data was all nan values
df = df.iloc[0:1133]

In [290]:
#writing to csv
df.to_csv("Property_Features_Data.csv",index = False)